# WES|WGSPipeline 

In [1]:
import os
import sys
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import PipelineManager
from util import DesignFileLoader

#s3 address of input files and output files
s3_input_files_address = "s3://ucsd-ccbb-interns/Mustafa/wgs_test/Sample_cDNA/mouse_samples"
s3_output_files_address = "s3://ucsd-ccbb-interns/Mustafa/wgs_test/Sample_cDNA/gatk_run"

## CFNCluster name
your_cluster_name = "mustafa7"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/keys/interns_oregon_key.pem"

## Project information
project_name = "mutect_output_check"

#bwa_gatk or bwa_mutect
workflow = "bwa_mutect"

#hg19 or GRCm38
genome = "GRCm38"

## If delete cfncluster after job is done.
delete_cfncluster= False

## Possible analysis_steps inputs for the two workflows. Order of input does not matter.

##bwa_gatk: "fastqc", "trim", "bwa", "sort", "dedup", "split", "postalignment", 
#"haplotype", "merge", "combine_vcf", "filter"

##bwa_mutect: "fastqc", "trim" , "bwa", "multiqc", "sort", "dedup", "split", "mutect", "bam_merge", "pair_vcf_merge"
analysis_steps = {
#                     "fastqc"
#                     ,"trim"
#                     ,"bwa"
#                     ,"multiqc"
#                     ,"sort"
#                     ,"dedup"
#                     ,"split"
#                     ,"postalignment"
#                     ,"haplotype"
#                     ,"mutect"
#                     ,"merge" 
#                     ,"bam_merge"
                    "pair_vcf_merge"
#                     ,"combine_vcf"
                    #"filter"
                }

#add design file path here
#examples in cirrus_root/data/cirrus-ngs/
design_file = "/home/mustafa/ccbb/cirrus-ngs/data/cirrus-ngs/mouse_test.txt"

print("variables set")

variables set


In [2]:
## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa7 does exist.
Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="34.214.180.149"
Output:"MasterPrivateIP"="172.31.33.31"
Output:"GangliaPublicURL"="http://34.214.180.149/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.33.31/ganglia/"

connecting
connected


In [9]:
sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("DNASeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, "NA", pairs_list)

[['SRR2473232_1.fastq.gz', 'SRR2473232_2.fastq.gz'], ['SRR616274_1.fastq.gz', 'SRR616274_2.fastq.gz']]
['groupA', 'groupA']
{'SRR2473232_1': 'SRR616274_1'}
making the yaml file...
copying yaml file to remote master node...
mutect_output_check.yaml
/shared/workspace/Pipelines/yaml_files/DNASeq/bwa_mutect
executing pipeline...


In [3]:
step = "all" #can be any step in analysis_steps or "all"
PipelineManager.check_status(ssh_client, step, "DNASeq", workflow, project_name, analysis_steps,verbose=True)

checking status of jobs...

Your project will go through the following steps:
	pair_vcf_merge

['2013', '0.55500', 'pair_merge', 'ec2-user', 'r', '01/19/2018', '19:50:10', 'all.q@ip-172-31-35-53.us-west-', '4']
The pair_vcf_merge step calls the pair_merge.sh script on the cluster
The pair_vcf_merge step is being executed
{'qw': 0, 'r': 1, 'dr': 0}
{'qw': (0, []), 'r': (1, ['\tone is currently running using 4 core(s) and was submitted 0 days, 0 hours, and 14 minutes ago']), 'dr': (0, [])}
There are 1 instances of the pair_vcf_merge step currently running
	one is currently running using 4 core(s) and was submitted 0 days, 0 hours, and 14 minutes ago

